In [1]:
# torch 함수 불러오기
# onnx로 변환하기
# 변환된 onnx 검증하기

In [1]:
import numpy as np

# torch 함수 불러오기
import torch
from add_ai5 import Encoder_n_Decoder
# onnx로 변환하기
import torch.onnx
# 변환된 onnx 검증하기
import onnx # 검증만
import onnxruntime # 실제 계산까지

In [2]:
# 함수 불러오기
file_name = "data/add_ai5"

F = torch.load(file_name + '.pt', weights_only=False, map_location='cpu') # cpu로 불러오기
F.eval() # dropout 끄기

print(F)

Encoder_n_Decoder(
  (encoder): Encoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (rnn): LSTM(11, 11, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (attention): Attention(
      (U): Linear(in_features=22, out_features=22, bias=True)
      (W): Linear(in_features=22, out_features=22, bias=True)
      (V): Linear(in_features=22, out_features=1, bias=True)
    )
    (rnn): LSTM(33, 11, batch_first=True, bidirectional=True)
    (f): Linear(in_features=22, out_features=12, bias=True)
  )
)


In [3]:
# onnx로 변환하기
x = torch.randint(0, 12, (1,7)).type(torch.long) # 입력값 더미 텐서
dynamic_axes = {'x' : {0 : 'b', 1 : 'f'}, 'y' : {0 : 'b'}} # 가변길이 차원 지정 dict

torch.onnx.export(
    F,
    x,
    file_name + '.onnx',
    export_params=True,
    opset_version=20,
    do_constant_folding=True,
    input_names=['x'],
    output_names=['y'],
    dynamic_axes=dynamic_axes
)

C:\ProgramData\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [4]:
# onnx 확인하기
onnx_F = onnx.load(file_name + '.onnx')

onnx.checker.check_model(onnx_F) #문제 없이 저장이 되었으면 오류 메세지 없음

In [5]:
# 실제 계산값 비교
onnx_F = onnxruntime.InferenceSession(file_name + '.onnx', providers=["CPUExecutionProvider"])
np_x = x.numpy()

y = F(x) # torch 계산 값

inputs = {onnx_F.get_inputs()[0].name : np_x}
outputs = onnx_F.run(None, inputs)
onnx_y = outputs[0] # onnx 계산 값

print(y.detach().numpy())
print(onnx_y)

np.testing.assert_allclose(y.detach().numpy(), onnx_y, rtol=1e-03, atol=1e-05) #계산 결과가 오차범위 내 차이 없으면 오류 메세지 없음

[[[-13.583315    34.72702     10.444136    -9.719709   -15.5479145
   -19.500704   -18.153072    -5.236818     7.994429    21.560867
     7.541352    -2.3782923 ]
  [ -6.397928     0.9416317  -13.6799555  -29.186928   -27.742537
   -13.507702     6.2380376   15.831399    21.01223     20.760471
    19.292858    -7.2076645 ]
  [ -4.946788     4.5097914    8.566231     7.4153886    3.2191849
    -0.41701412  -6.192226    -8.4336605   -4.781098     1.2866303
     2.7624612   -8.654132  ]
  [  8.594806    -3.6023984  -10.739      -18.167614   -18.337341
   -11.474478    -6.3194966    3.727161    11.1729145   12.714137
     8.067759   -11.955338  ]]]
[[[-13.583313    34.727024    10.444137    -9.719709   -15.547916
   -19.500704   -18.153065    -5.2368155    7.99443     21.560867
     7.54135     -2.3782911 ]
  [ -6.397927     0.94164395 -13.679943   -29.186918   -27.742538
   -13.507708     6.238034    15.831395    21.012226    20.760477
    19.29286     -7.2076626 ]
  [ -4.946787     4.509